In [ ]:
import sys
sys.path.append("..")
import json
from pathlib import Path
import scraper
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib import parse

In [ ]:
data_dir = Path("../data/")
format_dir = data_dir / "T20I"

In [ ]:
test = scraper.get_match_detail(1336039, 1336049)
test

In [ ]:
def save_detail(series_id, match_id, base_path):
    detail = scraper.get_match_detail(series_id, match_id)
    try:
        save_path = base_path / "{0[match][slug]}-{0[match][objectId]}".format(detail)
        save_path.mkdir(exist_ok=True)
        scraper.save_json(detail, save_path / "meta.json", debug=True)
    except Exception:
        print("failed for {} {} {}".format(series_id, match_id, base_path))

In [ ]:
# Then for each series we can get the match metadata
for year_path in [path for path in format_dir.iterdir()]:
    print(year_path)
    with ThreadPoolExecutor() as pool:
        futures=[]
        for series_path in year_path.iterdir():
            print(series_path)
            series_meta = scraper.load_json(series_path / "series_meta.json")
            season_meta = scraper.load_json(series_path / "season_meta.json")

            try:
                url = season_meta["links"][1]["href"]
                series_id = parse.parse_qsl(url)[0][1]
            except:
                series_id = series_meta["mappings"]["cricinfo"]
            
            try:
                event_urls = scraper.extract_items(scraper.json_resp("{}/{}".format(season_meta["$ref"], "events")))
                
                event_ids = scraper.extract_ids(event_urls)
            except:
                event_urls, event_ids = [], []

            matches_dir = series_path / "matches" 
            matches_dir.mkdir()

            for match_id, url in zip(event_ids, event_urls):            
                futures.append(pool.submit(save_detail, series_id, match_id, matches_dir))
        
        [r.result() for r in as_completed(futures)]
                
                    